In [1]:
import findspark
findspark.init()

#1a01e0e77d803e8e292dbb639aa77646d41947892fe6f227

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Files") \
    .master("spark://a92682f5ddcc:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/06 06:31:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from grandeInformacion.spark_utils import SparkUtils

logs = spark \
                .readStream \
                .format("text") \
                .option("cleanSource", "archive") \
                .option("sourceArchiveDir", "/home/jovyan/archive") \
                .load("/home/jovyan/notebooks/data/logs_output/")

In [4]:
from pyspark.sql.functions import split, trim


logs = logs.withColumn("value", trim(logs["value"]))

logs_df = logs.select(split(logs.value, "\\|").alias("log_array"))


logs_df = logs_df.withColumn("date", logs_df["log_array"].getItem(0).cast("string"))
logs_df = logs_df.withColumn("type", logs_df["log_array"].getItem(1).cast("string"))
logs_df = logs_df.withColumn("message", logs_df["log_array"].getItem(2).cast("string"))
logs_df = logs_df.withColumn("node", logs_df["log_array"].getItem(3).cast("string"))

logs_df.printSchema()


root
 |-- log_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- date: string (nullable = true)
 |-- type: string (nullable = true)
 |-- message: string (nullable = true)
 |-- node: string (nullable = true)



In [6]:
query = logs_df \
    .writeStream \
    .outputMode("append") \
    .trigger(processingTime='25 seconds') \
    .option("truncate", "false") \
    .format("console") \
    .start()

query.awaitTermination(200)

25/04/06 06:33:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ce916090-c4ee-4948-93cd-a8f9487f941c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/06 06:33:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|log_array                                                             |date               |type |message                  |node         |
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|[06/04/2025 06:33:07, ERROR, 500 Internal Server Error, server-node-2]|06/04/2025 06:33:07|ERROR|500 Internal Server Error|server-node-2|
|[06/04/2025 06:33:09, INFO, User login successful, server-node-1]     |06/04/2025 06:33:09|INFO |User login successful    |server-node-1|
|[06/04/2025 06:33:11, ERROR, 500 Internal Server Error, server-node-2]|06/04/2025 06:33:11|ERROR|500 Internal Server Error|server-node-2|
|[06/04/2025 06:33:13, ERROR, 500 Internal Server Error, server-node-

25/04/06 06:34:10 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921177.305525.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921177.305525.txt / skip moving file.
25/04/06 06:34:10 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921187.312382.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921187.312382.txt / skip moving file.
25/04/06 06:34:10 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921197.3260434.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921197.3260434.txt / skip moving file.
25/04/06 06:34:10 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921207.3316302.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921

-------------------------------------------
Batch: 5
-------------------------------------------
-------------------------------------------
Batch: 1
-------------------------------------------
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|log_array                                                             |date               |type |message                  |node         |
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|[06/04/2025 06:33:47, INFO, User login successful, server-node-1]     |06/04/2025 06:33:47|INFO |User login successful    |server-node-1|
|[06/04/2025 06:33:49, WARN, Disk usage 85%, server-node-1]            |06/04/2025 06:33:49|WARN |Disk usage 85%           |server-node-1|
|[06/04/2025 06:33:51, ERROR, 500 Internal Server Error, server-node-2]|06/04/2025 06:33:51|ERROR|500 Internal 

25/04/06 06:34:35 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921227.444926.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921227.444926.txt / skip moving file.
25/04/06 06:34:35 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921237.45004.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921237.45004.txt / skip moving file.
25/04/06 06:34:35 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921247.454793.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921247.454793.txt / skip moving file.


...

25/04/06 06:36:15 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921327.5146062.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921327.5146062.txt / skip moving file.
25/04/06 06:36:15 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921337.5210698.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921337.5210698.txt / skip moving file.
25/04/06 06:36:15 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921347.4970765.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921347.4970765.txt / skip moving file.


-------------------------------------------
Batch: 6
-------------------------------------------
-------------------------------------------
Batch: 10
-------------------------------------------
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|log_array                                                             |date               |type |message                  |node         |
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|[06/04/2025 06:35:57, WARN, Disk usage 85%, server-node-1]            |06/04/2025 06:35:57|WARN |Disk usage 85%           |server-node-1|
|[06/04/2025 06:35:59, ERROR, 500 Internal Server Error, server-node-2]|06/04/2025 06:35:59|ERROR|500 Internal Server Error|server-node-2|
|[06/04/2025 06:36:01, WARN, Disk usage 85%, server-node-1]            |06/04/2025 06:36:01|WARN |Disk usage 8

25/04/06 06:36:40 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921357.5025065.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921357.5025065.txt / skip moving file.
25/04/06 06:36:40 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743921367.5076702.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743921367.5076702.txt / skip moving file.


-------------------------------------------
Batch: 7
-------------------------------------------
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|log_array                                                             |date               |type |message                  |node         |
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|[06/04/2025 06:36:27, WARN, Disk usage 85%, server-node-1]            |06/04/2025 06:36:27|WARN |Disk usage 85%           |server-node-1|
|[06/04/2025 06:36:29, ERROR, 500 Internal Server Error, server-node-2]|06/04/2025 06:36:29|ERROR|500 Internal Server Error|server-node-2|
|[06/04/2025 06:36:31, INFO, User login successful, server-node-1]     |06/04/2025 06:36:31|INFO |User login successful    |server-node-1|
|[06/04/2025 06:36:33, WARN, Disk usage 85%, server-node-1]          

False

In [7]:
query.stop()

In [8]:
sc.stop()

25/04/06 06:37:05 ERROR MicroBatchExecution: Query [id = a0c86bf6-4d87-46b1-9ed7-72be13c5c6a8, runId = 0f8d452f-c7dd-4760-8c50-e955681567a5] terminated with error
java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.ja